<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/logo.png?raw=true" alt="2023年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# ゲノムアセンブル
　近年、シロイヌナズナやイネなどのようなモデル生物だけでなく、さまざまな生物でゲノム配列の整備（ゲノム解読）が進められています。解読されたゲノム配列は、進化学的研究や育種学的研究など、幅広い研究の基盤情報になります。

　次世代シーケンシング（ショートリードシーケンシング）や第三世代シーケンシング（ロングリードシーケンシング）をおこなうと、元々のゲノム配列に由来する、短い塩基配列（**リード、Read**）を多数得られます。ゲノム解読は、多数の短い配列をつなぎ、元のゲノム配列を復元する試みです。その復元処理は**ゲノムアセンブル（Genome assemble）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/assemble_overview.png?raw=true" height="100px" align="middle">

　今回の実習では、ショートリードやロングリードからゲノム配列を復元できるのかを見ていきます。また、ショートリードやロングリードの特徴についても勉強していきます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_and_long_reads.png?raw=true" height="100px" align="middle">



### サンプルデータのダウンロード
　この実習で使用するサンプルデータをダウンロードします。

　このテキストでは、[約40 kbのゲノム配列](https://raw.githubusercontent.com/CropEvol/lecture/refs/heads/master/textbook_2025/dataset/sample_genome.fa) から仮想的に得た以下のシーケンシングデータを使用します。
- ショートリードデータ
  - リードの量: ゲノムサイズの約80倍
  - リードの長さ: 各50 bp
  - データファイル:
    - `illumina_read1.fastq`
    - `illumina_read2.fastq`
- ロングリードデータ
  - リードの量: ゲノムサイズの約10倍
  - リードの長さ: 平均およそ3000 bp
  - データファイル:
    - `nanopore.fastq`
- オリジナルのゲノム配列（復元したゲノム配列を確認するために使用）
  - ゲノムの長さ: 約40 kb
  - データファイル:
    - `sample_genome.fa`


In [ ]:
%%bash
wget -q -O 'assemble_dataset.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2025/scripts/assemble_dataset.sh
bash assemble_dataset.sh
ls | grep -e 'fastq'

### ソフトウェアのダウンロード

　今回使用するソフトウェアとその実行プログラムをダウンロードをします。



In [ ]:
%%bash
wget -q -O 'assemble_softwares.sh' https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2025/scripts/assemble_softwares.sh
bash assemble_softwares.sh
ls tools/

### 今回の勉強内容

1. ショートリードを使ったゲノムアセンブル
 1. ショートリードを確認する
 1. ゲノムアセンブル
 1. ショートリードの特徴

1. ロングリードを使ったゲノムアセンブル
 1. ロングリードを確認する
 1. ゲノムアセンブル
 1. ロングリードの特徴

1. ゲノム配列を改良する






---
## 1. ショートリードを使ったゲノムアセンブル

　ショートリードのゲノムアセンブルをおこない、ゲノムの再構築を試みてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_read_assemble.png?raw=true" height="100px" align="middle">

### 1-1. ショートリードを確認する
　最初に、ゲノムアセンブルに使用するショートリードデータを確認してみましょう。DNA断片の両端をシーケンシングした**ペアエンド（Paired-end）**という形のリードデータを使用します。ペアエンドのデータは、通常2つのファイルに分かれており、それらを1組のデータとして同時に扱います。

使用するショートリード:
- Read1: /content/illumina_read1.fastq
- Read2: /content/illumina_read2.fastq
- リードの量: ゲノムサイズの約80倍
- リードの長さ: 各50 bp


<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/paired_end_reads.png?raw=true" height="200px" align="middle">

#### リードのデータファイル形式（FASTQ）

　リードのデータは**FASTQ**と呼ばれる形式でファイルに保存されています。FASTQ形式では、4行で1つのリードデータの情報を表しています。

```
@リードの名前
TGAAGCAGT..............(リードの塩基配列)
+
F1GCFCGGE..............(各塩基のクオリティ)
```

- 第1行（@からはじまる行）: リードの名前
- 第2行: リードの塩基配列
- 第3行（+からはじまる行）: 第2行と第4行の区切りのための行
- 第4行: 各塩基配列のクオリティ（ASCIIコードで書かれている）。1文字が何らかのクオリティスコア（数値）を示している。
 - 参考: https://bi.biopapyrus.jp/rnaseq/qc/fastq-quality-score.html



#### seqkitを使ったリードの統計情報確認

 　[seqkit](https://bioinf.shenwei.me/seqkit/)というソフトウェアを使って、ペアエンドのうちRead1の統計情報（リードの数やリードの長さなどの情報）を確認してみましょう。

```sh
# リードの統計情報の確認
seqkit stat 塩基配列データ
```

In [ ]:
%%bash
seqkit stat illumina_read1.fastq

### 実習1-1

　seqkitを使って、もう一方のリード（ペアエンドのRead2: illumina_read2.fastq）の統計情報を確認してください。

In [ ]:
%%bash
seqkit stat

### 1-2. ゲノムアセンブル

　今回、[Platanus](http://platanus.bio.titech.ac.jp/)というショートリードのゲノムアセンブルのためのソフトウェアを使って、ゲノム配列の復元を試みましょう。

　ここでは、全ショートリードデータを使わずに、seqkitを使って70％のショートリードをランダムサンプリングして、アセンブルに使います。  
<small>※ 実習1-2で、使用するリード量をより少なくしたり多くしたりしたときにどのような結果になるか確認してもらいます。</small>

　次のコードセルのプログラムで、ペアエンドショートリードのゲノムアセンブルをおこなってみてください。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/genome_assemble.png?raw=true" height="250px" align="middle">

In [ ]:
%%bash
## 使用するサンプルデータの割合（0.0~1.0のあいだで変更可能）
sampling_rate=0.7

## リードをサンプリング
seqkit sample -s 1 -p $sampling_rate illumina_read1.fastq > illumina_read1.subset.fastq
seqkit sample -s 1 -p $sampling_rate illumina_read2.fastq > illumina_read2.subset.fastq
## アセンブル
bash Platanus_assemble.sh illumina_read1.subset.fastq illumina_read2.subset.fastq


　アセンブルで得られたゲノム配列（**アセンブリ、Assembly**）を確認してみましょう。Colabサーバー上の次のファイルに保存されています。

/content/platanus_output/illumina_contig.fa

　アセンブリは、1本ではなく、複数本の配列に分かれており、元のゲノム配列を完全に再現できていません。アセンブル後の複数本に分かれた配列は**コンティグ（Contig）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/contigs.png?raw=true" height="100px" align="middle">

　seqkitを使って、コンティグの統計情報（配列数、配列の平均長、N50など）をみてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/about_n50.png?raw=true" height="100px" align="middle">


In [ ]:
%%bash
## オプション「-a」を追加すると、N50を含む情報を得られる
seqkit stat -a /content/platanus_output/illumina_contig.fa

### 実習1-2
　使用するリードの量（`sampling_rate`に与える数値）を変更して、アセンブル結果がどのように変わるか確認してください。

例:  
- `sampling_rate=0.5`にする
- `sampling_rate=1.0`にする

In [ ]:
%%bash
## 使用するサンプルデータの割合（0.0~1.0のあいだで変更可能）
sampling_rate=0.7

## リードをサンプリング
seqkit sample -s 1 -p $sampling_rate illumina_read1.fastq > illumina_read1.subset.fastq 2>/dev/null
seqkit sample -s 1 -p $sampling_rate illumina_read2.fastq > illumina_read2.subset.fastq 2>/dev/null
## アセンブル
bash Platanus_assemble.sh illumina_read1.subset.fastq illumina_read2.subset.fastq 2>/dev/null
## アセンブルの統計情報
seqkit stat -a /content/platanus_output/illumina_contig.fa

### 補足: リードの量とアセンブル結果

　実習1-2で見てもらったとおり、ゲノムアセンブルではリードの量が多いほど良いアセンブル結果が得られます。しかし、過剰にリード量を増やしたところで、下図のとおりある程度のところで頭打ちになってしまいます。

　必要なリードの量は、解析する対象の生物のゲノムサイズや倍数性などにも依存します。ショートリードでゲノムアセンブルする場合、ゲノムサイズの100倍以上のリードデータを使用することがほとんどです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/Assemble_figs/sampling_rate.png?raw=true" height="250px" align="middle">

### 1-3 ショートリードの特徴

　オリジナルのゲノム配列に対して、ショートリードとアセンブリを並べてみて、ショートリードやそれから作られるアセンブリの特徴を見てみましょう。なお、ゲノム配列にリード等の塩基配列を並べる操作を **アライメント（Alignment）** と言います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_read_alignment.png?raw=true" height="250px" align="middle">

In [ ]:
%%bash
## アセンブリをオリジナルのゲノム配列にアライメント
OUTPUT="illumina_assembly"
REF="sample_genome.fa"
CONTIG="/content/platanus_output/illumina_contig.fa"
bash Alignment_mm2.sh $REF $OUTPUT $CONTIG

## ショートリードをオリジナルのゲノム配列にアライメント
OUTPUT="illumina_alignment"
REF="sample_genome.fa"
READ1="illumina_read1.subset.fastq"
READ2="illumina_read2.subset.fastq"
bash Alignment_bwa.sh $REF $OUTPUT $READ1 $READ2


In [ ]:
## アライメント結果をみる
fa   = "sample_genome.fa"       #オリジナルのゲノム配列
bam1 = "illumina_assembly.bam"  #アセンブリのアライメント結果
bam2 = "illumina_alignment.bam" #ショートリードのアライメント結果

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": fa, "indexPath": fa+".fai", "id": fa })
aln1 = BamTrack({ "name": bam1, "path": bam1, "height":120 })
aln2 = BamTrack({ "name": bam2, "path": bam2, "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(aln1)
b.load_track(aln2)

### ショートリードゲノムアセンブルのまとめ

　ショートリードシーケンシングは、非常に多数の短い配列（リード）を得られます。塩基の読み間違いも少ない（シーケンシングエラー率も低い）です。

　生物のゲノムは、同じ塩基パターンの繰り返し（リピート）などを多く含んでいます。
ショートリードは、リード長よりも長いリピート配列に弱く、ショートリードだけでゲノムを再現するのはほぼ不可能です。データ量を増やしても、完全に再現できることはなく、どこかで頭打ちになります。一方で、ショートリードの強みは、シーケンシングエラーが少ないことです。そのため、精度の高い配列情報が得られます。

<small>※ロングリードに比べて、塩基あたりのシーケンシングコストがかなり低く、1回のシーケンシングで多量の塩基配列を得られることも強みです。</small>

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/short_reads_and_repeat.png?raw=true" height="250px" align="middle">

---
## 2. ロングリードを使ったゲノムアセンブル

　ショートリードシーケンシングは、50-300 bpの短い塩基配列を多量に得るシーケンシングです。近年、非常に長い塩基配列を得るシーケンシング技術（**ロングリードシーケンシング、Long read sequencing**）が発達してきました。Oxford Nanopore Technologies社のナノポアシーケンシングや、Pacific Biosciences社のPacBioシーケンシングがそれにあたります。

　現在では、ロングリードを使ったゲノムアセンブルが最もよくおこなわれています。ここでは、ナノポアシーケンシングによるロングリードからどのようなゲノム配列を復元できるかを見ていきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble.png?raw=true" height="100px" align="middle">

### 2-1. ロングリードを確認する

　ロングリードのシーケンスデータを見てみましょう。

使用するロングリード:
- ロングリード: /content/nanopore.fastq
- リードの量: ゲノムサイズの約10倍
- リードの長さ: 平均の長さが約3000 bp

　seqkitを使って、ロングリードの統計情報も確認してみましょう。

In [ ]:
%%bash
seqkit stat nanopore.fastq

### 2-2. ゲノムアセンブル
　ロングリードを使ったアセンブルでは、ロングリード用のソフトウェア[Flye](https://github.com/fenderglass/Flye)を使って、ゲノムアセンブルを試みてみましょう。

　ここでは、全ロングリードの内、70%のリードのみを使ってアセンブルしてみます。

<small>※ 実習2で、全ロングリードを使ったアセンブルをおこないます。</small>



In [ ]:
%%bash
## 使用するサンプルデータの割合（0.0~1.0のあいだで変更可能）
sampling_rate=0.7
## リードをサンプリング
seqkit sample -s 1 -p $sampling_rate nanopore.fastq > nanopore.subset.fastq 2>/dev/null
## アセンブル
bash Flye_assemble.sh nanopore.subset.fastq


　アセンブリは、次のファイルに保存されます。

/content/flye_output/assembly.fasta

　seqkitで、アセンブリの統計情報を確認してみましょう。

In [ ]:
%%bash
seqkit stat -a /content/flye_output/assembly.fasta

### 実習2
　使用するリードの量を`sampling_rate=1.0`に変更して、ゲノムアセンブル結果を確認してください。

In [ ]:
%%bash
## 使用するサンプルデータの割合（0.0~1.0のあいだで変更可能）
sampling_rate=0.7
## リードをサンプリング
seqkit sample -s 1 -p $sampling_rate nanopore.fastq > nanopore.subset.fastq 2>/dev/null
## アセンブル
bash Flye_assemble.sh nanopore.subset.fastq 2>/dev/null
## アセンブルの統計情報
seqkit stat -a /content/flye_output/assembly.fasta

### 2-3. ロングリードの特徴
　オリジナルのゲノム配列に、ロングリードやアセンブリをアライメントし、それらの特徴を見てみましょう。

In [ ]:
%%bash
## アセンブリをオリジナルのゲノム配列にアライメント
OUTPUT="nanopore_assembly"
REF="sample_genome.fa"
CONTIG="/content/flye_output/assembly.fasta"
bash Alignment_mm2.sh $REF $OUTPUT $CONTIG
s
## ロングリードをオリジナルのゲノム配列にアライメント
OUTPUT="nanopore_alignment"
REF="sample_genome.fa"
READ="nanopore.subset.fastq"
bash Alignment_bwa.sh $REF $OUTPUT $READ


In [ ]:
## アライメント結果をみる
fa   = "sample_genome.fa"        #オリジナルのゲノム配列
bam1 = "nanopore_assembly.bam"   #アセンブリのアライメント結果
bam2 = "nanopore_alignment.bam"  #ロングリードのアライメント結果

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": fa, "indexPath": fa+".fai", "id": fa })
aln1 = BamTrack({ "name": bam1, "path": bam1, "height":120 })
aln2 = BamTrack({ "name": bam2, "path": bam2, "viewAsPairs":True })
b = igv_notebook.Browser(ref)
b.load_track(aln1)
b.load_track(aln2)


### ロングリードゲノムアセンブリのまとめ

　ロングリードは、シーケンシングエラー率が高く、構築されたコンティグの塩基が正確ではない場合があります。しかし、ショートリードの場合とは異なり、ロングリードを使ったゲノムアセンブリは、非常に長いコンティグを生成できます。そのため、リピート配列を多く含む生物のゲノムアセンブリにおいて、ロングリードは主力的な役割を果たしています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble2.png?raw=true" height="250px" align="middle">

---
## 3. ゲノム配列を改良する

　リード数が多く、塩基情報が正確なショートリードと、長いコンティグを生成できるロングリードを組み合わせることで、より良いゲノム配列を得ることができます。

　先ほど得られたロングリードのゲノム配列（コンティグ）には、塩基情報が不正確そうな箇所がみられます。ここでは、ショートリードでその不正確性の補正を試みてみましょう。この補正処理は **ポリッシング（Polishing）** と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/polishing.png?raw=true" height="220px" align="middle">

### ショートリードを使った補正

In [ ]:
%%bash
CONTIG="/content/flye_output/assembly.fasta"
OUTPUT="assembly"
READ1="illumina_read1.fastq"
READ2="illumina_read2.fastq"

bash Polishing.sh $CONTIG $OUTPUT $READ1 $READ2

### 補正後のゲノム配列を確認する

　補正後のゲノム配列にショートリードをアライメントし、その結果をIGVで確認してみましょう。

In [ ]:
%%bash
## 補正前のアセンブリをオリジナルのゲノム配列にアライメント
REF="sample_genome.fa"
OUTPUT1="nanopore_assembly"
CONTIG1="/content/flye_output/assembly.fasta"
bash Alignment_mm2.sh $REF $OUTPUT1 $CONTIG1

## 補正後のアセンブリをオリジナルのゲノム配列にアライメント
OUTPUT2="nanopore_assembly_polished"
CONTIG2="/content/assembly_polished.fasta"
bash Alignment_mm2.sh $REF $OUTPUT2 $CONTIG2

In [ ]:
## アライメント結果をみる
fa   = "sample_genome.fa"
bam1 = "nanopore_assembly.bam"
bam2 = "nanopore_assembly_polished.bam"

## ---- 以下、変更不要 -----
import igv_notebook
from igv_prep import RefTrack, AnnotationTrack, BamTrack
bam_common_info = { "displayMode":"SQUISHED", "viewAsPairs":True, "height":100, "showCoverage":False }
igv_notebook.init()
ref = RefTrack({ "fastaPath": fa, "indexPath": fa+".fai", "id": fa })
aln1 = BamTrack({ "name": bam1, "path": bam1, "height":120 })
aln2 = BamTrack({ "name": bam2, "path": bam2, "height":120 })
b = igv_notebook.Browser(ref)
b.load_track(aln1)
b.load_track(aln2)


　今回、ロングリードのアセンブル結果をショートリードで補正しました。そのほかにも、ロングリードをショートリードで補正し、エラーの少ないロングリードにした後にアセンブルをおこなう方法や、ショートリードのアセンブリをロングリードでより長くする方法などがあります。

---
## まとめ

　今回、ショートリードとロングリードを使ってゲノムアセンブルをおこない、ゲノム配列の復元を試みました。

　ショートリードでは、正確なコンティグを得られる一方で、ゲノム全体を再現するほど充分長い配列は得られませんでした。要因としては、ゲノムに含まれるリピート配列の存在などが考えられます。

　ロングリードでは、非常に長いコンティグを得ることができました。しかし、エラー率が高いリードを使ったため、部分的には塩基情報が不正確なコンティグが生成されました。

　ショートリードとロングリードそれぞれの利点を活かすことで、より良いゲノム配列を構築することができます。今回の実習でも、ロングリードで得られた長いコンティグを、ショートリードの正確性で補い、高精度なゲノム配列を得ることができました。このようなロングリードとショートリードを組み合わせたゲノム解読は、現在よく使われています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/major_assemble_proc.png?raw=true" height="220px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>